In [1]:
# Imports

import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
classifiers = ["EDC", "random_forest", "logistic_regression", "svm_linear", "svm_rbf", "decision_tree"]
score_metric = "auc_score"
score_metric_index = 0 if score_metric == "auc_score" else 1

result_files = os.listdir("results")
results = []
for result_file in result_files:
    dataset, search_strategy, optimiser, random_seed = result_file.split(
        "."
    )[0].split("-")
    with open(f"results/{result_file}", "rb") as f:
        res = pickle.load(f)
    for elapsed_time, auc_score, accuracy_score in zip(res["elapsed_time"], res["auc_score"], res["accuracy_score"]):
        results.append(
            [
                dataset,
                "EDC",  # "classifier
                search_strategy,
                optimiser,
                elapsed_time,
                auc_score,
                accuracy_score,
                random_seed,
            ]
        )

result_files = os.listdir("comparison_results")
for result_file in result_files:
    dataset, classifier, random_seed = result_file.split(
        "."
    )[0].split("-")
    with open(f"comparison_results/{result_file}", "rb") as f:
        res = pickle.load(f)
    for elapsed_time, auc_score, accuracy_score in zip(res["elapsed_time"], res["auc_score"], res["accuracy_score"]):
        results.append(
            [
                dataset,
                classifier,  # "classifier
                None,
                None,
                elapsed_time,
                auc_score,
                accuracy_score,
                random_seed,
            ]
        )


df = pd.DataFrame(
    columns=[
        "dataset",
        "classifier",
        "search_strategy",
        "optimiser",
        "elapsed_time",
        "auc_score",
        "accuracy_score",
        "random_seed"
    ],
    data=results,
)

In [3]:
dataset_grouped = df.groupby(["dataset"])
all_scores = []
all_stds = []
for dataset_group in dataset_grouped:
    scores = {}
    stds = {}
    dataset = dataset_group[0][0]
    if dataset == "BANANA" or (dataset.startswith("AD0")):
        continue
    for classifier_group in dataset_group[1].groupby(["classifier"]):
        number_of_folds_done = len(classifier_group[1])

        classifier = classifier_group[0][0]
        mean_auc = classifier_group[1]["auc_score"].mean()
        std_auc = classifier_group[1]["auc_score"].std()
        mean_acc = classifier_group[1]["accuracy_score"].mean()
        std_acc = classifier_group[1]["accuracy_score"].std()
        scores[classifier] = (mean_auc, mean_acc)
        stds[classifier] = (std_auc, std_acc)
        


    # Get the ranks
    for classifier in classifiers:
        current_score = scores[classifier][score_metric_index]
        rank = 1
        for other_classifier in classifiers:
            if other_classifier == classifier:
                continue
            if scores[other_classifier][score_metric_index] > current_score:
                rank += 1
        scores[classifier] = (scores[classifier][0], scores[classifier][1], rank)
    

    print(f"{dataset}", end="&")
    if score_metric == "auc_score":
        # Print AUC scores
        for clf in classifiers:
            print(f"{scores[clf][0]:.4f} ($\\pm{stds[clf][0]:.2f}$)", end="&")

    if score_metric == "accuracy_score":
        # Print Acc
        for clf in classifiers:
            print(f"{scores[clf][1]:.4f} ($\\pm{stds[clf][1]:.2f}$)", end="&")
    
    # # Print rank
    # for clf in classifiers:
    #     print(f"{scores[clf][2]}", end="&")
    
    print(f"\b\\\\")

    all_scores.append(scores)
    all_stds.append(stds)
    # print(f"{dataset} & {scores['EDC'][0]:.4f} & {scores['random_forest'][0]:.4f} & {scores['EDC'][1]:.4f} & {scores['random_forest'][1]:.4f} \\\\")



# Get average ranks
average_ranks = {}
average_aucs = {}
average_accs = {}
for classifier in classifiers:
    average_ranks[classifier] = 0
    average_aucs[classifier] = 0
    average_accs[classifier] = 0
    for dataset in all_scores:
        if classifier not in dataset:
            continue
        average_ranks[classifier] += dataset[classifier][2]
        average_aucs[classifier] += dataset[classifier][0]
        average_accs[classifier] += dataset[classifier][1]
    average_ranks[classifier] /= len(all_scores)
    average_aucs[classifier] /= len(all_scores)
    average_accs[classifier] /= len(all_scores)

print("\\midrule")
print(f"Average Score&", end="")
for classifier in classifiers:
    print(f"{average_aucs[classifier]:.4f}", end="&")
print(f"\b\\\\")

print(f"Average Rank&", end="")
for classifier in classifiers:
    print(f"{average_ranks[classifier]:.2f}", end="&")
print(f"\b\\\\")

KeyError: 'EDC'